In [41]:
from sklearn.preprocessing import LabelEncoder
import re
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn

In [47]:
def createTrainingCorpus(corpusFile):
    import csv
    corpus=[]
    with open(corpusFile, 'rb') as csvfile:
        lineReader = csv.reader(csvfile,delimiter=',',quotechar="\"")
        for row in lineReader:
            corpus.append({"text":row[5],"label":int(row[0])})
    
    
    trainingData=[]
    for label in [0,4]:
        i=1
        for tweet in corpus:
            if tweet["label"]==label and i<=5000:
                trainingData.append(tweet)
                i+=1
    return trainingData

corpusFile="/Users/Josh/Desktop/Software Project/trainingCorpus/trainingData1600000.csv"
trainingData=createTrainingCorpus(corpusFile)

In [48]:


class PreProcessTweets:
    def __init__(self):
        self._stopwords=set(stopwords.words('english')+list(punctuation)+['rt', "'s", 'i'])
        
    def processTweets(self, list_of_tweets):
        
        processedTweets=[]
       
        for tweet in list_of_tweets:
            processedTweet={"text":self._processTweet(tweet["text"]),"label":tweet["label"]}
            if len(processedTweet["text"]) > 0:
                processedTweets.append(processedTweet)  
        return processedTweets
    
    def _processTweet(self, tweet):
        
        tweet=tweet.lower()
        
        tweet=re.sub('https?://[^\s]+','',tweet)
        
        tweet=re.sub(r'@[^\s]+','',tweet)
        
        tweet=re.sub(r'#([^\s]+)',r'\1',tweet)
        
        tweet=re.sub("[^a-z]", " ",tweet)
        
        tweet=word_tokenize(tweet)
        
        stemmer=PorterStemmer()
        tweet=[stemmer.stem(word) for word in tweet]
        stripper = lambda word: word.strip()
        tweet = list(map(stripper, tweet))
        tweet = filter(None, tweet)
        
        return [word for word in tweet if word not in self._stopwords]
        

In [49]:

tweetProcessor=PreProcessTweets()
ppTrainingData=tweetProcessor.processTweets(trainingData)

In [50]:

def storePPTrainingData(ppTrainingData,tweetDataFile):
    import csv
    with open(tweetDataFile,'wb') as csvfile:
        linewriter=csv.writer(csvfile,delimiter=',',quotechar="\"")
        convertLabels={4:"positive",0:"negative"}
        for tweet in ppTrainingData:
            try: 
                linewriter.writerow([_convertList(tweet["text"]), convertLabels[tweet["label"]]])
            except Exception,e:
                print e
                
def _convertList(tweetList):
    sentence=''
    for word in tweetList:
        sentence += word + ' '
    return sentence
                
tweetDataFile="/Users/Josh/Desktop/Software Project/trainingCorpus/processedCorpus.csv"
storePPTrainingData(ppTrainingData,tweetDataFile)